In [4]:
# 📦 1. Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

# 📄 2. Cargar datasets
df_sellin = pd.read_csv("sell-in.txt", sep="\t")
df_productos = pd.read_csv("tb_productos.txt", sep="\t")

# 📄 Leer lista de productos a predecir
with open("product_id_apredecir201912.TXT", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# 🧹 3. Preprocesamiento
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()
df_monthly_product['item_id'] = df_monthly_product['product_id']

# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)
ts_data = ts_data.fill_missing_values()

# ⚙️ 5. Entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS',
    eval_metric='MASE',
    verbosity=2
)

predictor.fit(
    ts_data,
    num_val_windows=2,
    time_limit=60 * 60,
    enable_ensemble=False,
    hyperparameters={
        "DeepAR": {"context_length": 12},
        "ETS": {},
        "AutoARIMA": {}
    }
)

# 📋 6. Leaderboard
print("\n🏆 Leaderboard:")
print(predictor.leaderboard())

# 🔮 7. Predicciones
forecast = predictor.predict(ts_data)
forecast_mean = forecast['mean'].reset_index()

# 🕒 Ver qué fechas están disponibles
fechas = forecast_mean['timestamp'].unique()
print("\n📅 Timestamps disponibles en el forecast:", fechas)

# Filtrar febrero si existe
resultado = forecast_mean[forecast_mean['timestamp'] == '2020-02-01']
if not resultado.empty:
    resultado = resultado[['item_id', 'mean']]
    resultado.columns = ['product_id', 'tn']

    # 💾 Guardar CSV
    resultado.to_csv("predicciones_febrero2020_autogluon_mejorado.csv", index=False)
    print("✅ Archivo guardado: predicciones_febrero2020_autogluon_mejorado.csv")

    # 📊 Gráfico de ejemplo
    ejemplo = resultado['product_id'].iloc[0]
    predictor.plot(item_id=str(ejemplo), ts_dataframe=ts_data)
    plt.title(f"Predicción para producto {ejemplo}")
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No se encontraron predicciones para 2020-02-01.")


No path specified. Models will be saved in: "AutogluonModels\ag-20250703_010341"
Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\AutogluonModels\ag-20250703_010341'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.23
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
GPU Count:          0
Memory Avail:       44.32 GB / 63.68 GB (69.6%)
Disk Space Avail:   1315.37 GB / 1862.26 GB (70.6%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': MASE,
 'freq': 'MS',
 'hyperparameters': {'AutoARIMA': {},
                     'DeepAR': {'context_length': 12},
                     'ETS': {}},
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_wind


🏆 Leaderboard:
       model  score_val  pred_time_val  fit_time_marginal  fit_order
0        ETS  -1.077332       3.668570           7.731666          1
1  AutoARIMA  -1.201972      28.769119          27.080944          2


Model not specified in predict, will default to the model with the best validation score: ETS



📅 Timestamps disponibles en el forecast: <DatetimeArray>
['2020-01-01 00:00:00', '2020-02-01 00:00:00']
Length: 2, dtype: datetime64[ns]
✅ Archivo guardado: predicciones_febrero2020_autogluon_mejorado.csv


TypeError: plot() got an unexpected keyword argument 'item_id'